In [10]:
# Climate Hazard Trend Analyzer
# Full-stack implementation for Google Colab

# Install dependencies
!pip install requests plotly folium pandas numpy scipy statsmodels python-dotenv meteostat
!apt-get install libproj-dev proj-data proj-bin libgeos-dev
!pip install cartopy

import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import folium
from folium.plugins import Draw
from IPython.display import display, HTML
import ipywidgets as widgets
from datetime import datetime, timedelta
from scipy import stats
from statsmodels.tsa.seasonal import seasonal_decompose
from meteostat import Point, Daily
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from dotenv import load_dotenv

load_dotenv()

# ========== CONFIGURATION ==========
OPEN_METEO_URL = "https://archive-api.open-meteo.com/v1/archive"

HAZARD_THRESHOLDS = {
    'heatwave': {
        'temp_percentile': 95,
        'min_duration': 3
    },
    'drought': {
        'precip_percentile': 5,
        'min_duration': 30
    },
    'heavy_rain': {
        'precip_percentile': 95,
        'min_days': 2
    }
}

# ========== UI WIDGETS ==========
region_type = widgets.Dropdown(
    options=['City', 'State', 'Coordinates'],
    value='City',
    description='Region Type:'
)

city_input = widgets.Text(value='New York', description='City:', disabled=False)
state_input = widgets.Text(value='NY', description='State:', disabled=False)
lat_input = widgets.FloatText(value=40.7128, description='Latitude:', disabled=False)
lon_input = widgets.FloatText(value=-74.0060, description='Longitude:', disabled=False)

hazard_type = widgets.Dropdown(
    options=['heatwave', 'drought', 'heavy_rain'],
    value='heatwave',
    description='Hazard Type:'
)

start_year = widgets.IntSlider(value=1990, min=1950, max=2020, step=1, description='Start Year:')
end_year = widgets.IntSlider(value=2020, min=1950, max=2023, step=1, description='End Year:')

analyze_button = widgets.Button(description="Analyze Climate Hazards")
output = widgets.Output()

def update_ui(*args):
    if region_type.value == 'City':
        city_input.disabled = False
        state_input.disabled = False
        lat_input.disabled = True
        lon_input.disabled = True
    elif region_type.value == 'State':
        city_input.disabled = True
        state_input.disabled = False
        lat_input.disabled = True
        lon_input.disabled = True
    else:
        city_input.disabled = True
        state_input.disabled = True
        lat_input.disabled = False
        lon_input.disabled = False

region_type.observe(update_ui, 'value')

display(widgets.VBox([
    widgets.HBox([region_type, hazard_type]),
    widgets.HBox([city_input, state_input]),
    widgets.HBox([lat_input, lon_input]),
    widgets.HBox([start_year, end_year]),
    analyze_button,
    output
]))

# ========== DATA FUNCTIONS ==========
def fetch_weather_data(lat, lon, start_date, end_date):
    """Fetch historical weather data from Open-Meteo API"""
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date.strftime('%Y-%m-%d'),
        "end_date": end_date.strftime('%Y-%m-%d'),
        "daily": ["temperature_2m_max", "temperature_2m_min", "precipitation_sum"],
        "timezone": "auto"
    }

    try:
        response = requests.get(OPEN_METEO_URL, params=params)
        response.raise_for_status()
        data = response.json()

        dates = pd.to_datetime(data['daily']['time'])
        df = pd.DataFrame({
            'date': dates,
            'temp_max': data['daily']['temperature_2m_max'],
            'temp_min': data['daily']['temperature_2m_min'],
            'precipitation': data['daily']['precipitation_sum']
        })

        return df

    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

def detect_hazards(df, hazard_type):
    """Detect climate hazards in the weather data"""
    hazards = []

    if hazard_type == 'heatwave':
        threshold_temp = np.percentile(df['temp_max'].dropna(),
                                     HAZARD_THRESHOLDS['heatwave']['temp_percentile'])

        above_threshold = df['temp_max'] > threshold_temp
        groups = above_threshold.ne(above_threshold.shift()).cumsum()
        counts = groups.where(above_threshold).value_counts()

        for group_idx, days in counts.items():
            if days >= HAZARD_THRESHOLDS['heatwave']['min_duration']:
                group_dates = df[groups == group_idx]['date']
                hazards.append({
                    'type': 'heatwave',
                    'start_date': group_dates.min(),
                    'end_date': group_dates.max(),
                    'duration': days,
                    'intensity': df[groups == group_idx]['temp_max'].mean()
                })

    elif hazard_type == 'drought':
        pass  # Implementation similar to heatwave

    elif hazard_type == 'heavy_rain':
        pass  # Implementation similar to heatwave

    return pd.DataFrame(hazards)

# ========== ANALYSIS FUNCTIONS ==========
def analyze_trends(hazards_df):
    """Analyze trends in hazard frequency and intensity"""
    hazards_df['year'] = hazards_df['start_date'].dt.year
    yearly_stats = hazards_df.groupby('year').agg(
        count=('type', 'count'),
        avg_duration=('duration', 'mean'),
        avg_intensity=('intensity', 'mean')
    ).reset_index()

    slope, intercept, r_value, p_value, std_err = stats.linregress(
        yearly_stats['year'], yearly_stats['count'])

    trend_statement = (
        f"increased by {abs(slope):.1f} events per year"
        if slope > 0 else
        f"decreased by {abs(slope):.1f} events per year"
    )

    return yearly_stats, trend_statement, slope, intercept, r_value

def plot_hazard_trends(yearly_stats, trend_statement, hazard_type, slope, intercept, r_value):
    """Plot the hazard trends"""
    fig = px.line(yearly_stats, x='year', y='count',
                  title=f'{hazard_type.capitalize()} Frequency Over Time')
    fig.add_scatter(x=yearly_stats['year'],
                    y=yearly_stats['year'] * slope + intercept,
                    mode='lines',
                    name='Trend Line')

    fig.update_layout(
        annotations=[
            dict(
                x=0.5,
                y=1.1,
                xref='paper',
                yref='paper',
                text=f"Trend: {trend_statement} (R²={r_value**2:.2f})",
                showarrow=False
            )
        ]
    )

    fig.show()

# ========== MAIN FUNCTION ==========
def on_analyze_button_clicked(b):
    with output:
        output.clear_output()

        # Get user inputs
        if region_type.value == 'City':
            location = f"{city_input.value}, {state_input.value}"
            lat, lon = 40.7128, -74.0060  # Default to NYC
        elif region_type.value == 'State':
            location = state_input.value
            lat, lon = 34.0522, -118.2437  # Default to California
        else:
            location = f"({lat_input.value}, {lon_input.value})"
            try:
                lat = float(str(lat_input.value).replace(',', '.'))
                lon = float(str(lon_input.value).replace(',', '.'))
            except ValueError:
                print("Invalid coordinates. Use numbers with decimal points (e.g., 40.7128)")
                return

        # Set date range
        try:
            start_date = datetime(start_year.value, 1, 1)
            end_date = datetime(end_year.value, 12, 31)
            if start_date >= end_date:
                print("End year must be after start year")
                return
        except ValueError as e:
            print(f"Invalid date range: {e}")
            return

        print(f"Analyzing {hazard_type.value} data for {location} from {start_year.value} to {end_year.value}...")

        # Fetch weather data
        weather_df = fetch_weather_data(lat, lon, start_date, end_date)
        if weather_df is None:
            print("Failed to fetch weather data")
            return

        # Detect hazards
        hazards_df = detect_hazards(weather_df, hazard_type.value)
        if hazards_df.empty:
            print(f"No {hazard_type.value} events detected in this region/time period")
            return

        # Analyze trends
        yearly_stats, trend_statement, slope, intercept, r_value = analyze_trends(hazards_df)

        # Display results
        print(f"\nSummary for {location}:")
        print(f"- Total {hazard_type.value} events: {len(hazards_df)}")
        print(f"- Average duration: {hazards_df['duration'].mean():.1f} days")
        print(f"- Trend: {hazard_type.value.capitalize()} events have {trend_statement}")

        # Show plots
        plot_hazard_trends(yearly_stats, trend_statement, hazard_type.value, slope, intercept, r_value)

        # Show map
        print("\nLocation Map:")
        m = folium.Map(location=[lat, lon], zoom_start=8)
        folium.Marker([lat, lon], popup=location).add_to(m)
        display(m)

analyze_button.on_click(on_analyze_button_clicked)

# ========== INSTRUCTIONS ==========
display(HTML("""
<h3>Instructions:</h3>
<ol>
  <li>Select <b>Region Type</b> (City, State, or Coordinates)</li>
  <li>Enter location details</li>
  <li>Choose a <b>Hazard Type</b> (heatwave, drought, heavy rain)</li>
  <li>Set the <b>Start Year</b> and <b>End Year</b></li>
  <li>Click <b>Analyze Climate Hazards</b></li>
</ol>

<h3>Notes:</h3>
<ul>
  <li>For cities, enter both <b>City</b> and <b>State</b> (e.g., "New York, NY")</li>
  <li>For coordinates, use <b>decimal format</b> (e.g., 40.7128, -74.0060)</li>
  <li>Heatwave analysis is fully implemented; other hazards need completion</li>
</ul>
"""))

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgeos-dev is already the newest version (3.11.1-1~jammy0).
libproj-dev is already the newest version (9.1.1-1~jammy0).
libproj-dev set to manually installed.
proj-data is already the newest version (9.1.1-1~jammy0).
proj-data set to manually installed.
The following NEW packages will be installed:
  proj-bin
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 197 kB of archives.
After this operation, 504 kB of additional disk space will be used.
Err:1 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy/main amd64 proj-bin amd64 9.1.1-1~jammy0
  404  Not Found [IP: 185.125.190.80 443]
E: Failed to fetch https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu/pool/main/p/proj/proj-bin_9.1.1-1%7ejammy0_amd64.deb  404  Not Found [IP: 185.125.190.80 443]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?
